inputs: data generated by 0_eda.ipynb

contents: model training scripts

output: the "best" model written to best_model/ (used in the streamlit dashboard)

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

df = pd.read_csv('data/model_ready.csv')
df.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,label
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,False
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,True
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,True
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,False
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,False


In [2]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.attribute_names]

class DataPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoders = {}

    def fit(self, X, y=None):
        # Convert all columns to string type to avoid type comparison issues
        X = X.astype(str)
        # Handle missing values and encode categorical columns
        categorical_cols = X.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col].fillna(X[col].mode()[0]))
            self.encoders[col] = le
        return self

    def transform(self, X):
        X = X.copy()
        # Convert all columns to string type to avoid type comparison issues
        X = X.astype(str)
        # Handle missing values in numeric columns
        numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
        for col in numeric_cols:
            X[col] = X[col].fillna(X[col].median())

        # Encode categorical columns
        for col, le in self.encoders.items():
            # Fit the encoder with all possible labels including those in the test set
            all_labels = np.unique(X[col].fillna(X[col].mode()[0]))
            le.classes_ = np.unique(np.concatenate((le.classes_, all_labels), axis=None))
            X[col] = le.transform(X[col].fillna(X[col].mode()[0]))
        return X

# Define the pipeline
pipeline = Pipeline([
    ('selector', DataFrameSelector(df.columns.drop('label').drop('id'))),
    ('preprocessor', DataPreprocessor()),
    ('classifier', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
])

# Split the data into features and target
X = df.drop('label', axis=1).drop('id', axis=1)
y = df['label']

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

/Users/uberj/miniconda/envs/capstone/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [17:16:14] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.86


In [4]:
import mlflow
import shutil

# Reset indices of X_train and y_train
X_train = X_train.reset_index(drop=True)
y_train = pd.Series(y_train).reset_index(drop=True)

# Sample 100 rows from X_train and y_train
X_train_sample = X_train.sample(n=100, random_state=42)
y_train_sample = y_train[X_train_sample.index]

# Remove the existing 'best_model' directory if it exists
shutil.rmtree('best_model', ignore_errors=True)

# Infer the signature of the model based on the sample data
signature = mlflow.models.signature.infer_signature(X_train_sample, y_train_sample)

# Save the trained model to the 'best_model' directory with the inferred signature
mlflow.sklearn.save_model(pipeline, 'best_model', signature=signature, input_example=X_train_sample)

/Users/uberj/miniconda/envs/capstone/lib/python3.11/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [5]:
X_train_sample

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal
669,40,Male,Hungary,non-anginal,130.0,215.0,False,normal,138.0,False,0.0,NaN,NaN,NaN
33,64,Male,Cleveland,typical angina,170.0,227.0,False,lv hypertrophy,155.0,False,0.6,flat,0.0,reversable defect
549,58,Male,VA Long Beach,atypical angina,126.0,0.0,True,normal,110.0,True,2.0,flat,NaN,NaN
199,62,Female,Cleveland,non-anginal,130.0,263.0,False,normal,97.0,False,1.2,flat,1.0,reversable defect
264,64,Female,Cleveland,non-anginal,140.0,313.0,False,normal,133.0,False,0.2,upsloping,0.0,reversable defect
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,53,Male,Hungary,atypical angina,120.0,NaN,False,normal,132.0,False,0.0,NaN,NaN,NaN
428,62,Male,Switzerland,asymptomatic,115.0,0.0,NaN,normal,128.0,True,2.5,downsloping,NaN,NaN
424,54,Male,VA Long Beach,non-anginal,NaN,203.0,False,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN
235,39,Male,Hungary,atypical angina,120.0,200.0,False,normal,160.0,True,1.0,flat,NaN,NaN
